# Resistive Temperature Device (RTD) Instrumentation 

In [1]:
import numpy as np

### Temperature Calculation
- 3850 device.

In [7]:
a = 3.9083e-3
b = -5.775e-7
rO = 100

In [8]:
tMax = 50
tMin = -50

In [9]:
rMax = rO*(1+a*tMax+b*tMax**2)
rMin = rO*(1+a*tMin+b*tMin**2)

print(f'Expected Resistance Range: {rMin:.2f} : {rMax:.2f}')

Expected Resistance Range: 80.31 : 119.40


In [10]:
def quad_solve(a,b,c):
    s1 = (-b + np.sqrt(b**2-4*a*c))/(2*a)
    s2 = (-b - np.sqrt(b**2-4*a*c))/(2*a)
    return s1,s2
    

In [11]:
print(f'Quad Solution for rMax: {quad_solve(b,a,1-rMax/rO)[0]:.2f}')
print(f'Quad Solution for rMin: {quad_solve(b,a,1-rMin/rO)[0]:.2f}')

Quad Solution for rMax: 50.00
Quad Solution for rMin: -50.00


In [12]:
tMaxLin = (rMax/rO-1)/a
tMinLin = (rMin/rO-1)/a
print(f'Linear Solution for rMax: {tMaxLin:.2f}')
print(f'Linear Solution for rMin: {tMinLin:.2f}')

Linear Solution for rMax: 49.63
Linear Solution for rMin: -50.37


### Measured Resistances

In [17]:
rIce = 101 # Ice Bath
rRoom = 108 # Room Temperature
rBody = 114.5  # Body Temperature
rO = 100 # Ideal RTD resistance at 0C

In [18]:
print(f'Quad Solution for rIce: {quad_solve(b,a,1-rIce/rO)[0]:.2f}')
print(f'Quad Solution for rRoom: {quad_solve(b,a,1-rRoom/rO)[0]:.2f}')
print(f'Quad Solution for rBody: {quad_solve(b,a,1-rBody/rO)[0]:.2f}')

Quad Solution for rIce: 2.56
Quad Solution for rRoom: 20.53
Quad Solution for rBody: 37.31


### ADC to Temperature Lookup Table

In [41]:
rSeries = 2650 # Series resistance for the RTD.
vDd = 3.3  # MCU Vdd
aV = 16 # Internal amplifier voltage gain.
t = np.arange(-50,50.5,0.5) # Desired temperature range.
adcBits = 12 # Number of ADC bits.

In [42]:
rRtd = rO*(1+a*t+b*t**2)

In [43]:
vIn = rRtd/(rRtd+rSeries) * vDd * aV

In [44]:
adcVal = (2**adcBits * vIn/vDd).astype('int')

In [47]:
tFxPt = (t*2).astype('int')  # Represent temp as a Q7.1 fixed point number.

In [57]:
# Output table C initialization code.
nTemp = len(tFxPt)
print(f'uint16_t rtdAdcToTemp[2][{nTemp}] = {{\n  {{',end='')

for i in range(nTemp):
    print(f' {adcVal[i]},',end='')
print('\b },\n  {',end='')

for i in range(nTemp):
    print(f' {tFxPt[i]},',end='')
print('\b }\n};')


uint16_t rtdAdcToTemp[2][201] = {
  { 1927, 1932, 1937, 1941, 1946, 1950, 1955, 1960, 1964, 1969, 1973, 1978, 1983, 1987, 1992, 1996, 2001, 2006, 2010, 2015, 2019, 2024, 2029, 2033, 2038, 2042, 2047, 2052, 2056, 2061, 2065, 2070, 2074, 2079, 2084, 2088, 2093, 2097, 2102, 2106, 2111, 2116, 2120, 2125, 2129, 2134, 2138, 2143, 2148, 2152, 2157, 2161, 2166, 2170, 2175, 2179, 2184, 2188, 2193, 2198, 2202, 2207, 2211, 2216, 2220, 2225, 2229, 2234, 2238, 2243, 2247, 2252, 2256, 2261, 2266, 2270, 2275, 2279, 2284, 2288, 2293, 2297, 2302, 2306, 2311, 2315, 2320, 2324, 2329, 2333, 2338, 2342, 2347, 2351, 2356, 2360, 2365, 2369, 2374, 2378, 2383, 2387, 2392, 2396, 2401, 2405, 2410, 2414, 2418, 2423, 2427, 2432, 2436, 2441, 2445, 2450, 2454, 2459, 2463, 2468, 2472, 2477, 2481, 2485, 2490, 2494, 2499, 2503, 2508, 2512, 2517, 2521, 2526, 2530, 2534, 2539, 2543, 2548, 2552, 2557, 2561, 2566, 2570, 2574, 2579, 2583, 2588, 2592, 2597, 2601, 2605, 2610, 2614, 2619, 2623, 2627, 2632, 2636, 2641, 2645, 26